In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
!pip install shap
import shap  # SHAP library for model explainability

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.9/532.9 kB 5.5 MB/s eta 0:00:00


In [4]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Fraud Detection in Deep Learning/data.csv')
df

,FLAG,Avg min between sent tnx,Avg min between received tnx,Time Diff between first and last (Mins),Sent tnx,Received Tnx,Number of Created Contracts,max value received,avg val received,avg val sent,total Ether sent,total ether balance,ERC20 total Ether received,ERC20 total ether sent,ERC20 total Ether sent contract,ERC20 uniq sent addr,ERC20 uniq rec token name,ERC20 most sent token type,ERC20_most_rec_token_type
0,Fraud,0.00,34682.010000,1.734100e+05,0,5,1,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000e-12,0.000000e+00,0.000000,0,1,-1,-1
1,Non - Fraud,202252.36,13562.510000,4.858798e+05,2,6,0,5.882285,2.358151,7.073171,14.146342,0.002562,1.092703e+03,0.000000e+00,0.000000,0,10,185,257
2,Non - Fraud,20807.97,23567.050000,3.550001e+05,8,8,0,15.252378,5.730538,5.730074,45.840593,0.003714,0.000000e+00,0.000000e+00,0.000000,0,0,4,3
3,Non - Fraud,80.96,155838.280000,4.683244e+05,10,3,0,2.148150,0.762823,0.450816,4.508160,-2.219690,0.000000e+00,0.000000e+00,0.000000,0,0,4,3
4,Fraud,0.00,1060.760000,7.936620e+03,1,7,0,1.493095,0.802857,5.619140,5.619140,0.000861,1.337000e+01,0.000000e+00,0.000000,0,1,1,74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19677,Non - Fraud,0.00,8925.590121,1.179567e+06,236,406,0,104.369013,0.000000,2.154970,1884.403523,791.729624,1.910098e+07,4.650915e+06,0.000000,0,1,268,404
19678,Non - Fraud,0.00,115.310274,7.850811e+03,10,5,1,0.000000,0.515251,2.304312,0.000000,-318.802857,4.191189e+06,6.781452e+05,1.886338,0,0,4,2
19679,Non - Fraud,149.14,8483.401999,7.276577e+05,0,8,0,48.964560,24.325597,0.000000,0.000000,-63.143938,0.000000e+00,8.429851e+05,9.653074,0,1,3,1
19680,Non - Fraud,3208.85,3590.754211,2.340333e+05,91,114,0,10.108521,7.043437,0.000000,0.000000,974.251706,0.000000e+00,6.665360e+06,4.677917,11,5,285,267


In [5]:
df['FLAG'] = df['FLAG'].map({'Fraud': 1, 'Non - Fraud': 0}).astype('float32')
df

,FLAG,Avg min between sent tnx,Avg min between received tnx,Time Diff between first and last (Mins),Sent tnx,Received Tnx,Number of Created Contracts,max value received,avg val received,avg val sent,total Ether sent,total ether balance,ERC20 total Ether received,ERC20 total ether sent,ERC20 total Ether sent contract,ERC20 uniq sent addr,ERC20 uniq rec token name,ERC20 most sent token type,ERC20_most_rec_token_type
0,1.0,0.00,34682.010000,1.734100e+05,0,5,1,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000e-12,0.000000e+00,0.000000,0,1,-1,-1
1,0.0,202252.36,13562.510000,4.858798e+05,2,6,0,5.882285,2.358151,7.073171,14.146342,0.002562,1.092703e+03,0.000000e+00,0.000000,0,10,185,257
2,0.0,20807.97,23567.050000,3.550001e+05,8,8,0,15.252378,5.730538,5.730074,45.840593,0.003714,0.000000e+00,0.000000e+00,0.000000,0,0,4,3
3,0.0,80.96,155838.280000,4.683244e+05,10,3,0,2.148150,0.762823,0.450816,4.508160,-2.219690,0.000000e+00,0.000000e+00,0.000000,0,0,4,3
4,1.0,0.00,1060.760000,7.936620e+03,1,7,0,1.493095,0.802857,5.619140,5.619140,0.000861,1.337000e+01,0.000000e+00,0.000000,0,1,1,74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19677,0.0,0.00,8925.590121,1.179567e+06,236,406,0,104.369013,0.000000,2.154970,1884.403523,791.729624,1.910098e+07,4.650915e+06,0.000000,0,1,268,404
19678,0.0,0.00,115.310274,7.850811e+03,10,5,1,0.000000,0.515251,2.304312,0.000000,-318.802857,4.191189e+06,6.781452e+05,1.886338,0,0,4,2
19679,0.0,149.14,8483.401999,7.276577e+05,0,8,0,48.964560,24.325597,0.000000,0.000000,-63.143938,0.000000e+00,8.429851e+05,9.653074,0,1,3,1
19680,0.0,3208.85,3590.754211,2.340333e+05,91,114,0,10.108521,7.043437,0.000000,0.000000,974.251706,0.000000e+00,6.665360e+06,4.677917,11,5,285,267


In [6]:
sp = StratifiedShuffleSplit(n_splits = 1, test_size = 0.2, random_state = 42)
for train_index, test_index in sp.split(df, df['FLAG']):
    train = df.loc[train_index]
    test = df.loc[test_index]

In [7]:
X_train = train.drop(['FLAG'],axis=1)

In [8]:
y_train = train['FLAG']

In [9]:
X_test = test.drop(['FLAG'],axis=1)

In [10]:
y_test = test['FLAG']

In [ ]:
# Define the MLP architecture
model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_split=0.2, epochs=200, batch_size=16, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'Test Accuracy: {accuracy*100:.2f}%')

# Predictions and Model Evaluation
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)  # Convert probabilities to binary output

# Print classification report
print(classification_report(y_test, y_pred))

# Confusion Matrix
print(confusion_matrix(y_test, y_pred))

# SHAP Values - Explain the model's predictions
explainer = shap.KernelExplainer(model.predict, X_train)
shap_values = explainer.shap_values(X_test, nsamples=10)  # nsamples is a parameter you can tweak

# Plot SHAP values
shap.summary_plot(shap_values, X_test, feature_names=feature_names)  # Replace feature_names with your actual features

Epoch 1/200
788/788 [==============================] - 10s 5ms/step - loss: 173437.0156 - accuracy: 0.7526 - val_loss: 8196.8105 - val_accuracy: 0.7812
Epoch 2/200
788/788 [==============================] - 7s 9ms/step - loss: 57773.0234 - accuracy: 0.7537 - val_loss: 18906.0664 - val_accuracy: 0.7774
Epoch 3/200
788/788 [==============================] - 6s 8ms/step - loss: 70505.2500 - accuracy: 0.7614 - val_loss: 18625.6484 - val_accuracy: 0.6078
Epoch 4/200
788/788 [==============================] - 5s 6ms/step - loss: 158116.0469 - accuracy: 0.7333 - val_loss: 130210.8281 - val_accuracy: 0.6135
Epoch 5/200
788/788 [==============================] - 4s 6ms/step - loss: 78166.3750 - accuracy: 0.7561 - val_loss: 7245.9253 - val_accuracy: 0.7599
Epoch 6/200
788/788 [==============================] - 3s 4ms/step - loss: 156016.9375 - accuracy: 0.7302 - val_loss: 4525.2920 - val_accuracy: 0.7367
Epoch 7/200
788/788 [==============================] - 3s 4ms/step - loss: 37312.4414 - accu

  0%|          | 0/3937 [00:00<?, ?it/s]

4921/4921 [==============================] - 9s 2ms/step


Regressors in active set degenerate. Dropping a regressor, after 2 iterations, i.e. alpha=1.541e-02, with an active set of 2 regressors, and the smallest cholesky pivot element being 2.220e-16. Reduce max_iter or increase eps parameters.
Regressors in active set degenerate. Dropping a regressor, after 3 iterations, i.e. alpha=7.707e-03, with an active set of 3 regressors, and the smallest cholesky pivot element being 2.220e-16. Reduce max_iter or increase eps parameters.
Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 5 iterations, alpha=7.104e-03, previous alpha=5.460e-03, with an active set of 4 regressors.


4921/4921 [==============================] - 9s 2ms/step


...

4921/4921 [==============================] - 11s 2ms/step


Regressors in active set degenerate. Dropping a regressor, after 1 iterations, i.e. alpha=5.894e-02, with an active set of 1 regressors, and the smallest cholesky pivot element being 8.429e-08. Reduce max_iter or increase eps parameters.
Regressors in active set degenerate. Dropping a regressor, after 2 iterations, i.e. alpha=2.947e-02, with an active set of 2 regressors, and the smallest cholesky pivot element being 2.220e-16. Reduce max_iter or increase eps parameters.
Regressors in active set degenerate. Dropping a regressor, after 2 iterations, i.e. alpha=2.947e-02, with an active set of 2 regressors, and the smallest cholesky pivot element being 8.429e-08. Reduce max_iter or increase eps parameters.
Regressors in active set degenerate. Dropping a regressor, after 2 iterations, i.e. alpha=2.456e-02, with an active set of 2 regressors, and the smallest cholesky pivot element being 2.220e-16. Reduce max_iter or increase eps parameters.
Regressors in active set degenerate. Dropping a 

4921/4921 [==============================] - 9s 2ms/step


Regressors in active set degenerate. Dropping a regressor, after 4 iterations, i.e. alpha=6.769e-03, with an active set of 4 regressors, and the smallest cholesky pivot element being 5.960e-08. Reduce max_iter or increase eps parameters.
Regressors in active set degenerate. Dropping a regressor, after 4 iterations, i.e. alpha=6.769e-03, with an active set of 4 regressors, and the smallest cholesky pivot element being 2.220e-16. Reduce max_iter or increase eps parameters.
Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 6 iterations, alpha=6.162e-03, previous alpha=4.548e-03, with an active set of 5 regressors.


4921/4921 [==============================] - 9s 2ms/step
